<a href="https://colab.research.google.com/github/CHGROSJEAN/2024_MLEES/blob/main/Projet/Projet_Charlotte_Grosjean_Stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Import libraries and data


Frist step, simple GMM for 2 stations : Grand-Vennes and Riand-Pré.

In [1]:
# Import
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
import os
from shapely.geometry import Point
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
import seaborn as sns

Download data sets

In [2]:
# Reading the Excel files
grandvennes = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/GrandVennes.xlsx")
riandpre = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/RiandPre.xlsx")
chandieu = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Chandieu.xlsx")
geopolis = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Geopolis.xlsx")
lexplore = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/LExplore.xlsx")
bethusy = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Bethusy.xlsx")
boisgentils = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/BoisGentils.xlsx")
elysee = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Elysee.xlsx")
pontaise = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Pontaise.xlsx")
rouvraie = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/Rouvraie.xlsx")
vclb = pd.read_excel("https://raw.githubusercontent.com/CHGROSJEAN/2024_MLEES/main/Projet/Stations/VersChezLesBlancs.xlsx")



# Displaying the first few rows of each dataset
print("Data from Grand-Vennes:")
print(grandvennes.head())

print("\nData from Riand-Pré:")
print(riandpre.head())

print("\nData from Chandieu:")
print(chandieu.head())

print("\nData from Geopolis:")
print(geopolis.head())

print("\nData from LExplore:")
print(lexplore.head())

print("\nData from Bethusy:")
print(bethusy.head())

print("\nData from Bois Gentils:")
print(boisgentils.head())

print("\nData from Elysée:")
print(elysee.head())

print("\nData from Pontaise:")
print(pontaise.head())

print("\nData from Rouvraie:")
print(rouvraie.head())

print("\nData from Vers-chez-les-Blanc:")
print(vclb.head())


Data from Grand-Vennes:
   Index             DateTime  Precipitation
0      1  07.01.2023 08:27:54              0
1      2  07.01.2023 08:30:54              0
2      3  07.01.2023 08:33:54              0
3      4  07.01.2023 08:36:54              0
4      5  07.01.2023 08:39:54              0

Data from Riand-Pré:
   Index             DateTime  Precipitation
0      1  09.11.2022 00:03:39              0
1      2  09.11.2022 00:06:39              0
2      3  09.11.2022 00:09:39              0
3      4  09.11.2022 00:12:39              0
4      5  09.11.2022 00:15:39              0

Data from Chandieu:
   Index             DateTime  Precipitation
0      1  30.11.2022 00:03:07              0
1      2  30.11.2022 00:06:07              0
2      3  30.11.2022 00:09:07              0
3      4  30.11.2022 00:12:07              0
4      5  30.11.2022 00:15:07              0

Data from Geopolis:
   Index             DateTime  Precipitation
0      1  11.11.2022 00:02:18              0
1      2  11

In [3]:
# Ensuring the 'DateTime' the column is of datetime type for both datasets
grandvennes['DateTime'] = pd.to_datetime(grandvennes['DateTime'], dayfirst=True)
riandpre['DateTime'] = pd.to_datetime(riandpre['DateTime'], dayfirst=True)
chandieu['DateTime'] = pd.to_datetime(chandieu['DateTime'], dayfirst=True)
geopolis['DateTime'] = pd.to_datetime(geopolis['DateTime'], dayfirst=True)
lexplore['DateTime'] = pd.to_datetime(lexplore['DateTime'], dayfirst=True)
bethusy['DateTime'] = pd.to_datetime(bethusy['DateTime'], dayfirst=True)
boisgentils['DateTime'] = pd.to_datetime(boisgentils['DateTime'], dayfirst=True)
elysee['DateTime'] = pd.to_datetime(elysee['DateTime'], dayfirst=True)
pontaise['DateTime'] = pd.to_datetime(pontaise['DateTime'], dayfirst=True)
rouvraie['DateTime'] = pd.to_datetime(rouvraie['DateTime'], dayfirst=True)
vclb['DateTime'] = pd.to_datetime(vclb['DateTime'], dayfirst=True)



# Setting the DateTime column as the index for easier time-based operations
grandvennes.set_index('DateTime', inplace=True)
riandpre.set_index('DateTime', inplace=True)
chandieu.set_index('DateTime', inplace=True)
geopolis.set_index('DateTime', inplace=True)
lexplore.set_index('DateTime', inplace=True)
bethusy.set_index('DateTime', inplace=True)
boisgentils.set_index('DateTime', inplace=True)
elysee.set_index('DateTime', inplace=True)
pontaise.set_index('DateTime', inplace=True)
rouvraie.set_index('DateTime', inplace=True)
vclb.set_index('DateTime', inplace=True)

# Analyse exploratoire

In [12]:
# List of DataFrames (stations)
stations_data = {
    "GrandVennes": grandvennes,
    "RiandPre": riandpre,
    "Chandieu": chandieu,
    "Geopolis": geopolis,
    "LExplore": lexplore,
    "Bethusy": bethusy,
    "BoisGentils": boisgentils,
    "Elysee": elysee,
    "Pontaise": pontaise,
    "Rouvraie": rouvraie,
    "VCLB": vclb
}

# Convert precipitation to millimeters
for station, df in stations_data.items():
    df["Precipitation_mm"] = df["Precipitation"] * 0.01

# 1. Descriptive statistics for each station
summary_stats = {}
for station, df in stations_data.items():
    summary_stats[station] = df["Precipitation_mm"].describe()

# Display descriptive statistics for each station
for station, stats in summary_stats.items():
    print(f"Statistics for {station}:")
    print(stats)
    print("\n")

Statistics for GrandVennes:
count    139377.000000
mean          0.005194
std           0.038283
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.410000
Name: Precipitation_mm, dtype: float64


Statistics for RiandPre:
count    173989.000000
mean          0.005403
std           0.031163
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.860000
Name: Precipitation_mm, dtype: float64


Statistics for Chandieu:
count    180825.000000
mean          0.001218
std           0.015785
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.680000
Name: Precipitation_mm, dtype: float64


Statistics for Geopolis:
count    166642.000000
mean          0.004637
std           0.042279
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           3.470000
Name: Precipitation_mm, dtype: float64

In [20]:
from matplotlib.backends.backend_pdf import PdfPages

# Prepare the descriptive statistics for each station
summary_stats_df = pd.DataFrame()

for station, df in stations_data.items():
    # Get the descriptive statistics for each station's precipitation data
    stats = df["Precipitation_mm"].describe()
    # Add the statistics for each station as a new row in the DataFrame
    summary_stats_df[station] = stats

# Transpose the dataframe so that each row represents a different statistic
summary_stats_df = summary_stats_df.T

# Save the table as a PDF
with PdfPages('descriptive_statistics.pdf') as pdf:
    fig, ax = plt.subplots(figsize=(12, 6))  # Create a figure and axis
    ax.axis('tight')
    ax.axis('off')

    # Create the table and add it to the PDF
    table = ax.table(cellText=summary_stats_df.values, colLabels=summary_stats_df.columns,
                    rowLabels=summary_stats_df.index, loc='center', cellLoc='center', bbox=[0, 0, 1, 1])

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)  # Scale the table to make it fit better
    pdf.savefig(fig)  # Save the current figure as a page in the PDF
    plt.close()


print(summary_stats_df)
print("Descriptive statistics table saved as 'descriptive_statistics.pdf'.")

                count      mean       std  min  25%  50%  75%   max
GrandVennes  139377.0  0.005194  0.038283  0.0  0.0  0.0  0.0  2.41
RiandPre     173989.0  0.005403  0.031163  0.0  0.0  0.0  0.0  1.86
Chandieu     180825.0  0.001218  0.015785  0.0  0.0  0.0  0.0  1.68
Geopolis     166642.0  0.004637  0.042279  0.0  0.0  0.0  0.0  3.47
LExplore     232860.0  0.014055  0.108224  0.0  0.0  0.0  0.0  3.27
Bethusy      158926.0  0.001225  0.015720  0.0  0.0  0.0  0.0  1.73
BoisGentils  200395.0  0.003525  0.024160  0.0  0.0  0.0  0.0  3.39
Elysee       183206.0  0.011667  0.063191  0.0  0.0  0.0  0.0  1.73
Pontaise     243072.0  0.007650  0.047594  0.0  0.0  0.0  0.0  5.37
Rouvraie     214518.0  0.001393  0.015295  0.0  0.0  0.0  0.0  2.34
VCLB         172991.0  0.000506  0.008940  0.0  0.0  0.0  0.0  1.13
Descriptive statistics table saved as 'descriptive_statistics.pdf'.


In [ ]:
# 2. Basic visualizations
# a) Histogram of precipitation for each station
for station, df in stations_data.items():
    plt.figure(figsize=(10, 6))
    sns.histplot(df["Precipitation_mm"], bins=50, kde=True, color="blue")
    plt.title(f"Histogram of precipitation for {station}")
    plt.xlabel("Precipitation (mm)")
    plt.ylabel("Frequency")
    plt.grid()
    plt.show()

In [ ]:

# b) Time series of precipitation for each station
for station, df in stations_data.items():
    plt.figure(figsize=(14, 7))
    plt.plot(df["DateTime"], df["Precipitation_mm"], label="Precipitation")
    plt.title(f"Time series of precipitation for {station}")
    plt.xlabel("Date")
    plt.ylabel("Precipitation (mm)")
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# c) Boxplot to compare precipitation across stations
all_data = pd.concat(stations_data.values(), ignore_index=True)
plt.figure(figsize=(12, 8))
sns.boxplot(x="Station", y="Precipitation_mm", data=all_data)
plt.title("Comparison of precipitation across stations")
plt.xlabel("Station")
plt.ylabel("Precipitation (mm)")
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
# 3. Missing values detection
missing_data = {station: df.isnull().sum() for station, df in stations_data.items()}
for station, missing in missing_data.items():
    print(f"Missing values for {station}:")
    print(missing)
    print("\n")


In [ ]:
# 4. Overall distribution of precipitation across all stations
plt.figure(figsize=(10, 6))
sns.histplot(all_data["Precipitation_mm"], bins=50, kde=True, color="orange")
plt.title("Distribution of precipitation across all stations")
plt.xlabel("Precipitation (mm)")
plt.ylabel("Frequency")
plt.grid()
plt.show()

In [ ]:

# Round down the DateTime to the previous minute (removes seconds, truncating)
grandvennes.index = grandvennes.index.floor('min')  # rounded to the nearest lower minute
riandpre.index = riandpre.index.floor('min')
bethusy.index = bethusy.index.floor('min')
boisgentils.index = boisgentils.index.floor('min')
chandieu.index = chandieu.index.floor('min')
elysee.index = elysee.index.floor('min')
lexplore.index = lexplore.index.floor('min')
pontaise.index = pontaise.index.floor('min')
rouvraie.index = rouvraie.index.floor('min')
vclb.index = vclb.index.floor('min')
geopolis.index = geopolis.index.floor('min')

# Resample both datasets to ensure they have data every 3 minutes (if necessary)
# Resampling to the closest 3-minute interval and filling missing values with NaN
grandvennes_resampled = grandvennes.resample('3min').mean()
riandpre_resampled = riandpre.resample('3min').mean()
bethusy_resampled = bethusy.resample('3min').mean()
boisgentils_resampled = boisgentils.resample('3min').mean()
chandieu_resampled = chandieu.resample('3min').mean()
elysee_resampled = elysee.resample('3min').mean()
lexplore_resampled = lexplore.resample('3min').mean()
pontaise_resampled = pontaise.resample('3min').mean()
rouvraie_resampled = rouvraie.resample('3min').mean()
vclb_resampled = vclb.resample('3min').mean()
geopolis_resampled = geopolis.resample('3min').mean()


# Align both datasets to the common time period (intersection of their timestamps)
start_time = max(grandvennes_resampled.index.min(), riandpre_resampled.index.min(),bethusy_resampled.index.min(),
                 boisgentils_resampled.index.min(),chandieu_resampled.index.min(),elysee_resampled.index.min(), lexplore_resampled.index.min(),
                 pontaise_resampled.index.min(),rouvraie_resampled.index.min(),vclb_resampled.index.min(),geopolis_resampled.index.min())


end_time = min(grandvennes_resampled.index.max(), riandpre_resampled.index.max(), bethusy_resampled.index.max(),
               boisgentils_resampled.index.max(),chandieu_resampled.index.max(),elysee_resampled.index.max(),lexplore_resampled.index.max(),
               pontaise_resampled.index.max(),rouvraie_resampled.index.max(),vclb_resampled.index.max(),geopolis_resampled.index.max())

grandvennes_aligned = grandvennes_resampled.loc[start_time:end_time]
riandpre_aligned = riandpre_resampled.loc[start_time:end_time]
bethusy_aligned = bethusy_resampled.loc[start_time:end_time]
boisgentils_aligned = boisgentils_resampled.loc[start_time:end_time]
chandieu_aligned = chandieu_resampled.loc[start_time:end_time]
elysee_aligned = elysee_resampled.loc[start_time:end_time]
lexplore_aligned = lexplore_resampled.loc[start_time:end_time]
pontaise_aligned = pontaise_resampled.loc[start_time:end_time]
rouvraie_aligned = rouvraie_resampled.loc[start_time:end_time]
vclb_aligned = vclb_resampled.loc[start_time:end_time]
geopolis_aligned = geopolis_resampled.loc[start_time:end_time]

# Drop rows with missing values from both datasets
grandvennes_aligned = grandvennes_aligned.dropna()
riandpre_aligned = riandpre_aligned.dropna()
bethusy_aligned = bethusy_aligned.dropna()
boisgentils_aligned = boisgentils_aligned.dropna()
chandieu_aligned = chandieu_aligned.dropna()
elysee_aligned = elysee_aligned.dropna()
lexplore_aligned = lexplore_aligned.dropna()
pontaise_aligned = pontaise_aligned.dropna()
rouvraie_aligned = rouvraie_aligned.dropna()
vclb_aligned = vclb_aligned.dropna()

# Now both datasets are aligned and cleaned. Viewing the cleaned and aligned data
print("Aligned and cleaned Grandvennes data:")
print(grandvennes_aligned.head())

print("Aligned and cleaned Riandpre data:")
print(riandpre_aligned.head())

print("Aligned and cleaned Bethusy data:")
print(bethusy_aligned.head())

print("Aligned and cleaned Boisgentils data:")
print(boisgentils_aligned.head())

print("Aligned and cleaned Chandieu data:")
print(chandieu_aligned.head())

print("Aligned and cleaned Elysee data:")
print(elysee_aligned.head())

print("Aligned and cleaned Lexplore data:")
print(lexplore_aligned.head())

print("Aligned and cleaned Pontaise data:")
print(pontaise_aligned.head())

print("Aligned and cleaned Rouvraie data:")
print(rouvraie_aligned.head())

print("Aligned and cleaned Vclb data:")
print(vclb_aligned.head())


Aligned and cleaned Grandvennes data:
                     Index  Precipitation
DateTime                                 
2023-01-07 08:27:00    1.0            0.0
2023-01-07 08:30:00    2.0            0.0
2023-01-07 08:33:00    3.0            0.0
2023-01-07 08:36:00    4.0            0.0
2023-01-07 08:39:00    5.0            0.0
Aligned and cleaned Riandpre data:
                       Index  Precipitation
DateTime                                   
2023-01-07 08:27:00  28489.0            0.0
2023-01-07 08:30:00  28490.0            0.0
2023-01-07 08:33:00  28491.0            0.0
2023-01-07 08:36:00  28492.0            0.0
2023-01-07 08:39:00  28493.0            0.0
Aligned and cleaned Bethusy data:
                       Index  Precipitation
DateTime                                   
2023-01-07 08:27:00  18409.0            0.0
2023-01-07 08:30:00  18410.0            0.0
2023-01-07 08:33:00  18411.0            0.0
2023-01-07 08:36:00  18412.0            0.0
2023-01-07 08:39:00  18413.